In [5]:
from pymatgen.core import Lattice, Structure 
import numpy as np 
from pymatgen.core.composition import Element, Composition
from pymatgen.core.periodic_table import Specie
import math
import random
from pymatgen.io.vasp.inputs import Poscar, Kpoints, Potcar,Incar
from pymatgen.io.vasp.outputs import Outcar 
from pymatgen.entries.computed_entries import ComputedStructureEntry
import os, json 

In [43]:
def create_computed_entry_from_outcar(poscar, outcar):
    """
    Create a ComputedEntry from the given OUTCAR file.

    Args:
        poscar (str): The path to the POSCAR file.
        outcar (str): The path to the OUTCAR file.

    Returns:
        ComputedStructureEntry: The computed entry object.

    Raises:
        FileNotFoundError: If the POSCAR or OUTCAR file is not found.
    """
    # need to test this with an outcar 
    outcar = Outcar(outcar)
    energy = outcar.final_energy_wo_entrp
    poscar = Poscar.from_file(poscar)
    structure = poscar.structure
    composition = structure.composition

    return ComputedStructureEntry(structure=structure, energy=energy, composition=composition)

def process_subfolder(subfolder_path):
    """
    Process a subfolder containing POSCAR and OUTCAR files to create a computed entry.

    Args:
        subfolder_path (str): The path to the subfolder containing the files.

    Returns:
        ComputedEntry: The computed entry created from the POSCAR and OUTCAR files.
    """
    # Check if POSCAR-original exists in the subdirectory
    poscar_original_path = os.path.join(subfolder_path, 'POSCAR-original')
    poscar_path = os.path.join(subfolder_path, 'POSCAR')
    outcar_path = os.path.join(subfolder_path, 'OUTCAR')

    if os.path.exists(poscar_original_path):
        return create_computed_entry_from_outcar(poscar_original_path, outcar_path)
    else:
        return create_computed_entry_from_outcar(poscar_path, outcar_path)


def create_entries_from_folder(folders):
    """
    Create entries from the given list of folders.

    Args:
        folders (list): A list of folder paths.

    Returns:
        list: A list of entries created from the subfolders in the given folders.
    """
    entries = []
    for folder in folders:
        for subfolder in os.listdir(folder):
            subfolder_path = os.path.join(folder, subfolder)
            if os.path.isdir(subfolder_path):
                entry = process_subfolder(subfolder_path)
                entries.append(entry)
        
    return entries

def closest_composition(comp, num_atoms, bal_element):
    """
    Calculate the closest composition of elements given a target number of atoms,
    adjusting the balance element to handle any shortfall or excess.
    
    Parameters:
    comp (dict): A dictionary representing the composition of elements, where the keys are the element symbols and the values are the fractions.
    num_atoms (int): The target number of atoms.
    bal_element (str): The symbol of the element used to adjust the balance so that the total number of atoms is correct.

    Returns:
    tuple: A tuple containing two dictionaries. The first dictionary represents the number of atoms for each element, with adjustments made through the balance element. The second dictionary represents the actual fractions of each element, rounded to 5 decimal places.
    """
    # Normalize the composition fractions
    total_fraction = sum(comp.values())
    normalized_comp = {element: fraction / total_fraction for element, fraction in comp.items()}
    
    # Calculate initial atoms for each element, excluding the balance element, rounding to nearest integer.
    atoms = {}
    for element, fraction in normalized_comp.items():
        if element != bal_element:
            atoms[element] = round(fraction * num_atoms)
    
    # Calculate the number of atoms assigned so far and adjust the balance element accordingly.
    assigned_atoms = sum(atoms.values())
    atoms[bal_element] = num_atoms - assigned_atoms
    
    # Recalculate the actual fractions to ensure they sum to 1
    actual_fractions = {element: round(atoms[element] / num_atoms, 5) for element in atoms}
    
    return atoms, actual_fractions



def create_cca_primitive(comp_list, a, prim = True):
    """
    Create a CCA (Complex Concentrated Alloy) primitive structure using the given atom dictionary and lattice constant.

    Parameters:
        atom_dict (dict): A dictionary containing the count of each atom in the supercell you eventually want.
        a (float): The lattice constant of the cubic cell.

    Returns:
        pymatgen.core.structure.Structure: The CCA supercell structure.

    """
    direct_coords = [[0,0,0]]
    if prim:
        test_bcc = Structure(Lattice.cubic(a), comp_list, direct_coords)
    else:
        test_bcc = Structure.from_spacegroup("Im-3m",Lattice.cubic(a), comp_list, direct_coords)
    return test_bcc

In [3]:
# list of folders 
folders = ['/Users/myless/Packages/structure_maker/VCrTiWZr_Data/CE_Vasp_Jobs','/Users/myless/Packages/structure_maker/VCrTiWZr_Data/CE_Vasp_Jobs_T3']


In [23]:
entries = create_entries_from_folder(folders)

Full Formula (Zr2 Ti6 V48 Cr4 W4)
Reduced Formula: ZrTi3V24(CrW)2
abc   :  10.426946  10.426946  10.426946
angles: 109.471221 109.471221 109.471221
pbc   :       True       True       True
Sites (64)
  #  SP           a         b         c
---  ----  --------  --------  --------
  0  Zr    0.74258   0.996069  0.247875
  1  Zr    0.999138  0.501467  0.75016
  2  Ti    0.751984  0.002558  0.498964
  3  Ti    0.248567  0.252343  0.998494
  4  Ti    0.505337  0.497119  0.005943
  5  Ti    0.497521  0.250775  0.251121
  6  Ti    0.497026  0.247485  0.746856
  7  Ti    0.250955  0.750398  0.743823
  8  V     0.743079  0.997038  0.989426
  9  V     0.498903  0.999628  0.000264
 10  V     0.749132  0.248759  0.000837
 11  V     0.500453  0.253268  0.995178
 12  V     0.74359   0.493761  3.2e-05
 13  V     0.50028   0.999549  0.25355
 14  V     0.755146  0.252299  0.254655
 15  V     0.006585  0.996185  0.004041
 16  V     0.741288  0.746926  0.989998
 17  V     0.2551    0.010936  0.253373
 18

In [24]:
# save the entries as a json file 
with open('vcrtiwzr_entries.json', 'w') as f:
    f.write(json.dumps([entry.as_dict() for entry in entries]))

In [44]:
# create a prim entry

# create a primitive structure
a = 3.01 
x_cr = 0.02
x_ti = 0.02
x_zr = 0.02
x_w = 0.02
x_v = round(1 - x_cr - x_ti - x_w - x_zr,5)
composition = {'V': x_v, 'Cr': x_cr, 'Ti': x_ti, 'W': x_w, 'Zr': x_zr}
num_atoms = 64
atom_dict, actual_comp = closest_composition(composition, num_atoms=num_atoms, bal_element='V')
dos_prim_struct = create_cca_primitive([actual_comp], a,prim=False)

In [45]:
print(composition)

{'V': 0.92, 'Cr': 0.02, 'Ti': 0.02, 'W': 0.02, 'Zr': 0.02}


In [46]:
print(actual_comp)

{'Cr': 0.01562, 'Ti': 0.01562, 'W': 0.01562, 'Zr': 0.01562, 'V': 0.9375}


In [47]:
print(dos_prim_struct)

Full Formula (Zr0.03124 Ti0.03124 V1.875 Cr0.03124 W0.03124)
Reduced Formula: Zr0.03124Ti0.03124V1.875Cr0.03124W0.03124
abc   :   3.010000   3.010000   3.010000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (2)
  #  SP                                                a    b    c
---  ----------------------------------------------  ---  ---  ---
  0  Zr:0.016, Ti:0.016, V:0.938, Cr:0.016, W:0.016  0    0    0
  1  Zr:0.016, Ti:0.016, V:0.938, Cr:0.016, W:0.016  0.5  0.5  0.5


In [48]:
prim_struct = create_cca_primitive([actual_comp], a, prim=True)


In [49]:
print(prim_struct)

Full Formula (Zr0.01562 Ti0.01562 V0.9375 Cr0.01562 W0.01562)
Reduced Formula: Zr0.01562Ti0.01562V0.9375Cr0.01562W0.01562
abc   :   3.010000   3.010000   3.010000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (1)
  #  SP                                                a    b    c
---  ----------------------------------------------  ---  ---  ---
  0  Zr:0.016, Ti:0.016, V:0.938, Cr:0.016, W:0.016    0    0    0


In [50]:
prim_path = 'v1_6cr1_6ti1_6w1_6zr_prim_struct.json'
entry_path = '/Users/myless/Packages/structure_maker/Entries'
prim_entry_path = os.path.join(entry_path,prim_path)
with open(prim_entry_path, 'w') as f:
    f.write(json.dumps(prim_struct.as_dict()))

In [51]:
prim_path = 'v1_6cr1_6ti1_6w1_6zr_prim_struct_dos.json'
entry_path = '/Users/myless/Packages/structure_maker/Entries'
prim_entry_path = os.path.join(entry_path,prim_path)
with open(prim_entry_path, 'w') as f:
    f.write(json.dumps(dos_prim_struct.as_dict()))